In [1]:
import h2o

In [2]:
from h2o.estimators.deeplearning import H2OAutoEncoderEstimator

In [6]:
import data.data_wrangling as dw

import numpy as np
import pandas as pd

import os

import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

from pyspark.sql.functions import avg, array

from pyspark.sql.functions import first

from pyspark.sql.functions import percent_rank

from pyspark.sql.window import Window

from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import svm
from pyspark import SparkContext

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import savefig
from sklearn.ensemble import IsolationForest

import numpy as np
import pandas as pd

import os

import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

from pyspark.sql.functions import avg

from pyspark.sql.functions import first

from pyspark.sql.functions import percent_rank

from pyspark.sql.window import Window

from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import svm
from pyspark import SparkContext

from mpl_toolkits import mplot3d
from statsmodels.graphics.mosaicplot import mosaic

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

import os

import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

from pyspark.sql.functions import avg

from pyspark.sql.functions import first

from pyspark.sql.functions import percent_rank

from pyspark.sql.window import Window

from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import svm
from pyspark import SparkContext

In [ ]:
# Read in data and get column names

data = pd.read_csv('../predict_income.csv')

data.shape

# data with outlier detection values

data_w_outlier_val = data.copy()

# outcome variable label

outcome = 'income'

# id column label

id_col = 'id'

# list of categorical columns

# obtained from kaggle website
cat_col = ['workclass', 'education',
           'marital-status', 'occupation', 
           'relationship',
           'race', 'sex', 
           'native-country', 'income']

# categorical columns without outcome variable

cat_col_wo_outcome = [col for col in cat_col if col != outcome]

# cat cols without outcome and two category values

cat_col_more_than_2 = list()
for col in cat_col:
    if data[col].nunique() > 2:
        cat_col_more_than_2.append(col)

cat_col_2_or_fewer = list(set(cat_col) - set(cat_col_more_than_2))

# list of non-id numeric columns

num_col = [col for col in data.columns if col not in cat_col and col != id_col]

# Convert categorical columns to "categorical" in pandas

data_with_cat = dw.convert_col_to_cat(data, cat_col)

data_with_cat.head()

# equal width discretization of numeric columns

data_with_cat = dw.convert_num_to_obj(data_with_cat, num_col)

# Attribute value frequency

## read in pandas df as spark df

spark = SparkSession.builder.getOrCreate()

data_cat_only_spark_df = spark.createDataFrame(data_with_cat)

for column in data_cat_only_spark_df.columns:
    if column not in ['id', 'income']:
        data_cat_only_spark_df = data_cat_only_spark_df.withColumn(column, count(column).over(Window.partitionBy(column)))

avg_cols = udf(lambda array: sum(array)/len(array), DoubleType())

wanted_cols = [col for col in data_cat_only_spark_df.columns if col not in ['id', 'income']]

data_cat_only_spark_df_with_avf = data_cat_only_spark_df.withColumn("avf", avg_cols(array(*wanted_cols)))

# ## count frequency for each category

# def count_freq_for_cat(df):
#     counts_dict = dict()
#     for col in df.select_dtypes(['category']):
#         counts_dict[col] = df[col].value_counts().to_dict()
#     return counts_dict

# counts_dict = count_freq_for_cat(data_with_cat)

# ## use map in pandas

# def map_freq_to_value(df, counts_dict):
#     df_with_freq = pd.DataFrame()
#     for col in df:
#         if str(df[col].dtypes) == 'category' and col != outcome:
#             df_with_freq[col] = df[col].map(counts_dict[col])
#         else:
#             df_with_freq[col] = df[col]
#     return df_with_freq

# data_num_to_cat_with_freq = map_freq_to_value(data_with_cat, counts_dict)

# data_num_to_cat_with_freq.head()

# data_num_to_cat_with_freq['avf'] = data_num_to_cat_with_freq.drop(columns=[outcome, id_col]).apply(np.sum, axis=1) / 13

data_cat_only_pd_with_avf = data_cat_only_spark_df_with_avf.toPandas()

data_cat_only_pd_with_avf[['id', 'avf', outcome]].head()

## add avf to data_w_outlier_val

data_w_outlier_val = data_w_outlier_val.merge(data_cat_only_pd_with_avf[['id', 'avf']], on='id')

# Local Outlier Factor

data_all_num = dw.convert_cat_to_numeric(data, cat_col_more_than_2, cat_col_2_or_fewer)

## drop unwanted variables

X = data_all_num.drop(columns=[outcome])

## fit predict local outlier factor

lof = LocalOutlierFactor(n_jobs=-1)

y_pred = lof.fit_predict(X.drop(columns=id_col))

len(lof.negative_outlier_factor_)

lof.negative_outlier_factor_ * -1

## make copy of X dataframe

X_lof = X.copy()

## add lof values to X dataframe

X_lof['lof'] = lof.negative_outlier_factor_ * -1

X_lof.head()

## add lof to data_w_outlier_val

data_w_outlier_val = data_w_outlier_val.merge(X_lof[['id', 'lof']], on='id')

data_w_outlier_val.head(100)

# one-class svm

## copy X dataframe to X_svm

X_svm = X.copy()

## standardize data for svm

X_standardized = dw.standardize_numeric_variables(X_svm.drop(columns=id_col), X_svm.columns[1:])

X_standardized.head()

## instantiate svm class

clf = svm.OneClassSVM()

## fit the data

clf.fit(X_standardized)

## get the predicted values

X_svm['svm'] = clf.decision_function(X_standardized)

# add svm results to data_w_outlier_val

data_w_outlier_val = data_w_outlier_val.merge(X_svm[['id', 'svm']], on='id')

data_w_outlier_val.head(100)

# isolation forest

X_standardized.shape

clf = IsolationForest(max_samples=100)
clf.fit(X_standardized)

# predictions
y_pred_train = clf.predict(X_standardized)

data_w_outlier_val['iso_for'] = clf.decision_function(X_standardized)

### The anomaly score of the input samples. The lower, the more abnormal. Negative scores represent outliers, positive scores represent inliers.

# autoencoder

data_all_num = dw.convert_cat_to_numeric(data, cat_col_more_than_2, cat_col_2_or_fewer)

num_data_standardized = dw.standardize_numeric_variables(data_all_num, data_all_num.columns)

X_train = num_data_standardized.drop(columns=['id', 'income'])

input_dim = X_train.shape[1]
encoding_dim = 14
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

nb_epoch = 100
batch_size = 32
autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
history = autoencoder.fit(X_train, X_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_train, X_train),
                    verbose=1,
                    callbacks=[checkpointer, tensorboard]).history

autoencoder = load_model('model.h5')

predictions = autoencoder.predict(X_train)

mse = np.mean(np.power(X_train - predictions, 2), axis=1)

sns.distplot(mse)

mse.describe()

data_w_outlier_val['autoencoder'] = mse

# Anomaly Detection Scores

def color(s):
    row_color_list = list()

    # avf
    if s.avf < 1000:
        row_color_list.extend(['background-color: red'])
    elif s.avf >= 1000 and s.avf < 1200:
        row_color_list.extend(['background-color: yellow'])
    elif s.avf >= 1200:
        row_color_list.extend(['background-color: green'])

    # lof
    if s.lof <= 1.05:
        row_color_list.extend(['background-color: green'])
    elif s.lof > 1.05 and s.lof < 1.3:
        row_color_list.extend(['background-color: yellow'])
    elif s.lof >= 1.3:
        row_color_list.extend(['background-color: red'])

    # svm
    if s.svm < -100:
        row_color_list.extend(['background-color: red'])
    elif s.svm >= -100 and s.svm < 100:
        row_color_list.extend(['background-color: yellow'])
    elif s.svm >= 100:
        row_color_list.extend(['background-color: green'])

    # iso_for
    if s.iso_for < -0.1:
        row_color_list.extend(['background-color: red'])
    elif s.iso_for >= -0.1 and s.iso_for < 0.1:
        row_color_list.extend(['background-color: yellow'])
    elif s.iso_for >= 0.1:
        row_color_list.extend(['background-color: green'])

    # autoencoder
    if s.autoencoder > 0.396917:
        row_color_list.extend(['background-color: red'])
    elif s.autoencoder <= 0.396917 and s.autoencoder > 0.206993:
        row_color_list.extend(['background-color: yellow'])
    elif s.autoencoder <= 0.206993:
        row_color_list.extend(['background-color: green'])

    return row_color_list

result_df = data_w_outlier_val.loc[:, ['id', 'income', 'avf', 'lof', 'svm', 'iso_for', 'autoencoder']]

result_df_style = result_df.sort_values(by='income',
                                        ascending=False).iloc[:1000, :1000].style.apply(color,
                                                                                        subset=['avf',
                                                                                               'lof',
                                                                                               'svm',
                                                                                               'iso_for',
                                                                                               'autoencoder'],
                                                                                        axis=1)

result_df_style

# 3d plots

%matplotlib notebook

fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection='3d')

# Data for three-dimensional scattered points
zdata = data_all_num.loc[data_all_num[outcome] == 0, 'hours-per-week']
xdata = data_all_num.loc[data_all_num[outcome] == 0, 'capital-loss']
ydata = data_all_num.loc[data_all_num[outcome] == 0, 'capital-gain']
ax.scatter3D(xdata, ydata, zdata, cmap=data_all_num[outcome], label='<=50K', alpha=0.5)
zdata = data_all_num.loc[data_all_num[outcome] == 1, 'hours-per-week']
xdata = data_all_num.loc[data_all_num[outcome] == 1, 'capital-loss']
ydata = data_all_num.loc[data_all_num[outcome] == 1, 'capital-gain']
ax.scatter3D(xdata, ydata, zdata, cmap=data_all_num[outcome], label='>50K', alpha=0.5)
ax.set_xlabel('capital-loss')
ax.set_ylabel('capital-gain')
ax.set_zlabel('hours-per-week')
ax.legend()

%matplotlib notebook

fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection='3d')

# Data for three-dimensional scattered points
zdata = data_all_num.loc[data_all_num[outcome] == 0, 'hours-per-week']
xdata = data_all_num.loc[data_all_num[outcome] == 0, 'age']
ydata = data_all_num.loc[data_all_num[outcome] == 0, 'capital-gain']
ax.scatter3D(xdata, ydata, zdata, cmap=data_all_num[outcome], label='<=50K', alpha=0.5)
zdata = data_all_num.loc[data_all_num[outcome] == 1, 'hours-per-week']
xdata = data_all_num.loc[data_all_num[outcome] == 1, 'age']
ydata = data_all_num.loc[data_all_num[outcome] == 1, 'capital-gain']
ax.scatter3D(xdata, ydata, zdata, cmap=data_all_num[outcome], label='>50K', alpha=0.5)
ax.set_xlabel('age')
ax.set_ylabel('capital-gain')
ax.set_zlabel('hours-per-week')
ax.legend()

# generalized pairs plot

data_cat_and_num = dw.convert_col_to_cat(data, cat_col)

data_cat_and_num.dtypes

data_cat_and_num.drop(columns='id', inplace=True)

data_cat_num_subset = data_cat_and_num[['age', 'workclass', 'education', 'occupation', 'capital-gain']]

col_names = data_cat_num_subset.columns

import itertools
col_pairs = [col for col in itertools.product(col_names, repeat=2)]

col_pairs

count = 0
fig, axes = plt.subplots(nrows=5, ncols=5)
for i, ax_row in enumerate(axes):
    for j, ax_col in enumerate(axes):
        pair = col_pairs[count]
        dtype_pair = list()
        dtype_pair.append(str(data_cat_num_subset[pair[0]].dtypes))
        dtype_pair.append(str(data_cat_num_subset[pair[1]].dtypes))
        print(dtype_pair)
        count += 1

sns.boxplot(x='workclass',y='age',data=data_cat_and_num.loc[:, ['workclass', 'age']], ax=ax)
fig.show()

fig, ax = plt.subplots()
sns.boxplot(x='workclass',y='age',data=data_cat_and_num.loc[:, ['workclass', 'age']], ax=ax)
fig.show()

fig, ax = plt.subplots()
sns.scatterplot(x='age',y='capital-gain',data=data_cat_and_num, ax=ax)
fig.show()

fig, ax = plt.subplots()
mosaic(data_cat_and_num, ['education', 'marital-status'], ax=ax, labelizer = lambda x: "")
fig.show()

fig, ax = plt.subplots()
sns.distplot(a=data_cat_and_num['age'],  ax=ax)
fig.show()

fig, ax = plt.subplots()
sns.countplot(x='race', data=data_cat_and_num,  ax=ax)
fig.show()